In [217]:
import numpy as np
import pandas as pd
import sys

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

sys.path.append("/Users/szagar/ZTS/Dropbox/Business/ats/Code/Model/v3")

from base import Base, Session

from dow_optimization import DowOptimization

In [218]:
dow_dir = '/Users/szagar/ZTS/Dropbox/Business/ats/Data/Queue/dowOptResults'
ef_dir = '/Users/szagar/ZTS/Dropbox/Business/ats/Data/Queue/efOptResults'

In [222]:
def queryStrategy(id):
    return session.query(Strategy).filter(Strategy.id == id).one_or_none()
def getStrategy(id):
    return session.query(Strategy).filter(Strategy.id == id).one_or_none()

In [ ]:
def getOptimization(id):
    return session.query(EfOptimization).filter(EfOptimization.id == id).one_or_none()

In [220]:
def queryDowResults():
    q = (
        session.query(DowOptimization)
        .filter(DowOptimization.status == "Opt")
        .filter(DowOptimization.status_state == "done")
        .all()
    )
    return q

In [ ]:
def strategyBaseLine(strat):
    pass


In [221]:
def update_strategy(strat):
    updates = {"dow_opt": strat[0].dow_opt+1}
    try:
        session.query(Strategy).filter(Strategy.id == strat[0].id).update(updates)
        session.commit()
    except:
        session.rollback()
        raise

In [188]:
def createStrategy(strat,params):
    new_rec = Strategy(
        parent_id=strat.id,
        sec_id=strat.sec_id,
        symbol=strat.symbol,
        oos_start_dt=strat.oos_start_dt,
        oos_end_dt=strat.oos_end_dt,
        status="new",
        status_state="pending",
        proto_id = strat.id,
        cand_id = strat.cand_id,
        session_id = strat.session_id,
        timeframe_1 = strat.timeframe_1,
        timeframe_1_unit = strat.timeframe_1_unit,
        timeframe_2 = strat.timeframe_2,
        timeframe_2_unit = strat.timeframe_2_unit,
        max_days_back = strat.max_days_back,
        oos_start_dt = strat.oos_start_dt,
        oos_end_dt = strat.oos_end_dt,
        use_second_data = strat.use_second_data,
        lsb = strat.lsb,
        trades_per_day = strat.trades_per_day,
        day_swing = strat.day_swing,
        poi_switch = strat.poi_swing,
        poi_n1 = strat.poi_n1,
        natr = strat.natr,
        fract = strat.fract,
        filter_1_switch = strat.filter_1_switch,
        filter_1_n1 = strat.filter_1_n1,
        filter_1_n2 = strat.fileter_1_n2,
        filter_2_switch = strat.filter_2_switch,
        filter_2_n1 = strat.filter_2_n1
        filter_2_n2 = strat.filter_2_n2,
        t_segment = strat.t_segment,
        sl_switch = strat.sl_switch,
        stop_loss = strat.stop_loss,
        pt_switch = strat.pt_switch,
        profit_target = strat.profit_target,
        fitness_function = strat.fitness_function,
        #input_vars = 
        reopt_period_days = strat.reopt_period_days,
        next_reopt_dt = strat.next_reopt_dt,
        next_reopt_start_dt = strat.next_reopt_start_dt,
        next_reopt_end_dt = strat.next_reopt_end_dt,
        oos_param_history = strat.oos_param_history,
        comm_entry_dol = strat.comm_entry_dol,
        comm_exit_dol = strat.comm_exit_dol,
        slippage_entry_dol = strat.slippage_entry_dol,
        slippage_exit_dol = strat.slippage_exit_dol,
        parent_id = strat.parent_id,
        strat_type = strat.strat_type

    )
    session.add(new_rec)
    session.commit()
    return 0


In [216]:
def analyseDOWoopt():
    for opt in queryDowResults():
        #/Users/szagar/ZTS/Dropbox/Business/ats/Data/Queue/DowOptResults/dow_opt_1.txt
        opt_report = f'{dow_dir}/dow_opt_{opt.id}.txt'
        #print(opt.id,opt.symbol,opt_report)
        df = pd.read_csv(opt_report,sep='\t', encoding = "utf-16" )
        df['npdd'] = df['All: Net Profit'] / -df['All: Max Intraday Drawdown']
        xdays = df[df['npdd']<1].shape[0]
        print(opt.id,opt.symbol,opt_report,xdays)
        if xdays in (0,5):
            return
        #print(f'Strategy: {opt.strat_id}')
        #print(df[df['npdd']>1][['Test','All: % Profitable','All: Total Trades','npdd']])
        #strat = getStrategy(opt.strat_id)
        #strat.ef_filters += 
        #strat.ef_params += 
        #createStrategy(strat)
analyseDOWoopt()

1 @RTY.D /Users/szagar/ZTS/Dropbox/Business/ats/Data/Queue/dowOptResults/dow_opt_1.txt 2
2 @RTY.D /Users/szagar/ZTS/Dropbox/Business/ats/Data/Queue/dowOptResults/dow_opt_2.txt 1
3 @RTY.D /Users/szagar/ZTS/Dropbox/Business/ats/Data/Queue/dowOptResults/dow_opt_3.txt 1
4 @ES.D /Users/szagar/ZTS/Dropbox/Business/ats/Data/Queue/dowOptResults/dow_opt_4.txt 2
5 @ES.D /Users/szagar/ZTS/Dropbox/Business/ats/Data/Queue/dowOptResults/dow_opt_5.txt 1
6 @RTY.D /Users/szagar/ZTS/Dropbox/Business/ats/Data/Queue/dowOptResults/dow_opt_6.txt 3
7 @RTY.D /Users/szagar/ZTS/Dropbox/Business/ats/Data/Queue/dowOptResults/dow_opt_7.txt 2
8 @RTY.D /Users/szagar/ZTS/Dropbox/Business/ats/Data/Queue/dowOptResults/dow_opt_8.txt 2


In [ ]:
def analyseExtFilterOpt():
    for opt in queryExtFilterOptResults():
        opt_report = f'{ef_dir}/ef_opt_{opt.id}.txt'
        df = pd.read_csv(opt_report,sep='\t', encoding='utf-16')
        df['npdd'] = df['All: Net Profit'] / -df['All: Max Intraday Drawdown']
        strat = getStrategy(opt.strat_id)
        if criteria_passed(df,strat):
            opt_params = getOptimization(opt.id)
            createStrategy(strat,opt_params)

In [186]:
db = Session()
db_url = os.environ["DB_HEROKU"]

engine = create_engine(db_url)
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)
session = DBSession()

In [50]:
for test_id in [4]:
    opt_report = f'{dir}/dow_opt_{test_id}.txt'
    print(opt_report)
    df = pd.read_csv(opt_report,sep='\t', encoding = "utf-16" )

/Users/szagar/ZTS/Dropbox/Business/ats/Data/Queue/DowOptResults/dow_opt_4.txt


In [51]:
import os
for filename in os.listdir(dir):
    df = pd.read_csv(opt_report,sep='\t', encoding = "utf-16" )

In [53]:
df

,s_245_oos_dow_opt: DOW,Test,All: Net Profit,All: Gross Profit,All: Gross Loss,All: Total Trades,All: % Profitable,All: Winning Trades,All: Losing Trades,All: Max Winning Trade,...,All: Max Intraday Drawdown,All: ProfitFactor,All: Max Contracts Held,All: Required Account Size,All: Return on Account,All: TS Index,All: Expectancy Score,All: Pessimistic Return On Capital,All: Perfect Profit Correlation,All: Robustness Index
0,5,5,4050.0,21400.0,-17350.0,144,61.805556,89,53,1150.0,...,-3987.5,1.233429,1,3987.5,101.567398,90.394984,0.389931,0.041648,3048.838624,0.0
1,4,4,14325.0,28312.5,-13987.5,130,60.000000,78,50,1200.0,...,-3537.5,2.024129,1,3537.5,404.946997,315.858657,2.066709,0.130974,13806.844758,0.0
2,3,3,3675.0,25700.0,-22025.0,130,50.769231,66,63,1650.0,...,-4125.0,1.166856,1,4125.0,89.090909,58.800000,0.255137,0.032864,2891.874787,0.0
3,2,2,6312.5,26837.5,-20525.0,147,52.380952,77,69,1262.5,...,-3762.5,1.307552,1,3762.5,167.774086,129.186047,0.747113,0.057250,5131.902502,0.0
4,1,1,-2925.0,15787.5,-18712.5,121,45.454546,55,65,1175.0,...,-6237.5,0.843687,1,6237.5,-46.893788,-25.791583,-0.627688,-0.027328,-2662.034250,0.0


In [140]:
test_id = 4
opt_report = f'{dir}/dow_opt_{test_id}.txt'
df= pd.read_csv(opt_report,sep='\t', encoding = "utf-16" )
df.columns
#pd.read_csv(opt_report,sep='\t', encoding = "utf-16" )['All: % Profitable','All: Total Trades','All: Net Profit'/'All: Max Intraday Drawdown']

Index(['s_245_oos_dow_opt: DOW', 'Test', 'All: Net Profit',
       'All: Gross Profit', 'All: Gross Loss', 'All: Total Trades',
       'All: % Profitable', 'All: Winning Trades', 'All: Losing Trades',
       'All: Max Winning Trade', 'All: Max Losing Trade',
       'All: Avg Winning Trade', 'All: Avg Losing Trade',
       'All: Win/Loss Ratio', 'All: Avg Trade', 'All: Max Consecutive Winners',
       'All: Max Consecutive Losers', 'All: Avg Bars in Winner',
       'All: Avg Bars in Loser', 'All: Max Intraday Drawdown',
       'All: ProfitFactor', 'All: Max Contracts Held',
       'All: Required Account Size', 'All: Return on Account', 'All: TS Index',
       'All: Expectancy Score', 'All: Pessimistic Return On Capital',
       'All: Perfect Profit Correlation', 'All: Robustness Index'],
      dtype='object')

In [141]:
df

,s_245_oos_dow_opt: DOW,Test,All: Net Profit,All: Gross Profit,All: Gross Loss,All: Total Trades,All: % Profitable,All: Winning Trades,All: Losing Trades,All: Max Winning Trade,...,All: Max Intraday Drawdown,All: ProfitFactor,All: Max Contracts Held,All: Required Account Size,All: Return on Account,All: TS Index,All: Expectancy Score,All: Pessimistic Return On Capital,All: Perfect Profit Correlation,All: Robustness Index
0,5,5,4050.0,21400.0,-17350.0,144,61.805556,89,53,1150.0,...,-3987.5,1.233429,1,3987.5,101.567398,90.394984,0.389931,0.041648,3048.838624,0.0
1,4,4,14325.0,28312.5,-13987.5,130,60.000000,78,50,1200.0,...,-3537.5,2.024129,1,3537.5,404.946997,315.858657,2.066709,0.130974,13806.844758,0.0
2,3,3,3675.0,25700.0,-22025.0,130,50.769231,66,63,1650.0,...,-4125.0,1.166856,1,4125.0,89.090909,58.800000,0.255137,0.032864,2891.874787,0.0
3,2,2,6312.5,26837.5,-20525.0,147,52.380952,77,69,1262.5,...,-3762.5,1.307552,1,3762.5,167.774086,129.186047,0.747113,0.057250,5131.902502,0.0
4,1,1,-2925.0,15787.5,-18712.5,121,45.454546,55,65,1175.0,...,-6237.5,0.843687,1,6237.5,-46.893788,-25.791583,-0.627688,-0.027328,-2662.034250,0.0


* Gross Profit < 0
* Profit Factor < 1
* Return on Account < 0
* TSI < 0

s_245_oos_dow_opt: DOW                  int64
Test                                    int64
All: Net Profit                       float64
All: Gross Profit                     float64
All: Gross Loss                       float64
All: Total Trades                       int64
All: % Profitable                     float64
All: Winning Trades                     int64
All: Losing Trades                      int64
All: Max Winning Trade                float64
All: Max Losing Trade                 float64
All: Avg Winning Trade                float64
All: Avg Losing Trade                 float64
All: Win/Loss Ratio                   float64
All: Avg Trade                        float64
All: Max Consecutive Winners            int64
All: Max Consecutive Losers             int64
All: Avg Bars in Winner                 int64
All: Avg Bars in Loser                  int64
All: Max Intraday Drawdown            float64
All: ProfitFactor                     float64
All: Max Contracts Held           

In [223]:
#Bitwise
session.query(User).filter(somecolumn.op('&')(1) == 1)

NameError: name 'User' is not defined

In [236]:
s = 'a{asd gh'; s1=''
if '{' in s: s,s1 = s.split('{',2) 
len(s),len(s1)

(1, 6)